<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize input text with padding and truncation
text = "Deep learning transformers are powerful."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Forward pass through the model
outputs = model(**inputs)

# Extract the last hidden states
last_hidden_states = outputs.last_hidden_state

# Print the shape of the last hidden states
print("Last hidden states shape:", last_hidden_states.shape)

# Optional: Extract the [CLS] token representation for sentence-level tasks
cls_representation = last_hidden_states[:, 0, :]  # First token (CLS token)
print("CLS Token Representation Shape:", cls_representation.shape)

# Example: Process input with a longer sequence for comparison
long_text = "Transformers have revolutionized NLP and are being applied to many other fields like vision, speech, and biology."
long_inputs = tokenizer(long_text, return_tensors="pt", padding=True, truncation=True)
long_outputs = model(**long_inputs)
long_hidden_states = long_outputs.last_hidden_state
print("Long text last hidden states shape:", long_hidden_states.shape)